# 在链上使用记忆

1. 作为 chain 参数

In [ ]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

#自定义模板
template = """你是一个可以和人类对话的机器人.
{chat_history}
人类:{human_input}
机器人:"""

prompt= PromptTemplate(
    template=template,
    input_variables=["chat_history", "human_input"],
)
memory = ConversationBufferMemory(
    memory_key="chat_history",
)
llm = OpenAI(
    temperature=0,
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)


chain.predict(human_input="我最新喜欢我的世界这个游戏，你还记得我叫什么吗？")

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content="你好，我是一个可以和人类对话的机器人",
            role="system",
        ),
        MessagesPlaceholder(
            variable_name="chat_history",
        ),
        HumanMessagePromptTemplate.from_template(
            "{human_input}"
        ),
    ]
)

print(prompt.format(human_input="你好",chat_history=[]))

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)

chain.predict(human_input="我叫tomie，我是一个AI应用程序猿")

2. ConversationChain

自定义模块

In [ ]:
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory

llm=OpenAI(
    temperature=0,
)
memory = ConversationBufferMemory(
    memory_key="history",
    return_messages=True,
)
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True,
)

chain.predict(input="帮我做个一日游攻略")


In [ ]:
#自定义一下，对其进行覆盖
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm=OpenAI(
    temperature=0,
)

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
当前对话:
{history}
Human:{input}
AI助手:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["history", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(
        ai_prefix="AI助手",
        return_messages=True,
    ),
    prompt=prompt,
    verbose=True,
)

chain.predict(input="你知道我叫什么名字吗?")

3. 同一个链合并使用多个memory

In [ ]:
from langchain.chains import ConversationChain
from langchain.llms import OpenAI
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    CombinedMemory
)
from langchain.prompts import PromptTemplate

llm = OpenAI(
    temperature=0,
)

#使用CoversationSummaryMemory对对话进行总结
summay = ConversationSummaryMemory(
    llm=llm,
    input_key="input"
)
#使用ConversationBufferMemory对对话进行缓存
cov_memory = ConversationBufferMemory(
    memory_key="history_now",
    input_key="input",
)

memory = CombinedMemory(
    memories=[summay, cov_memory],
)

TEMPLATE = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'.
之前的对话摘要:
{history}
当前对话:
{history_now}
Human:{input}
AI："""

prompt = PromptTemplate(
    template=TEMPLATE,
    input_variables=["history", "history_now", "input"],
)

chain = ConversationChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
)


In [ ]:
chain.run("那ETH呢？")

4. 多参数链增加记忆

In [ ]:
from langchain.embeddings.openai import  OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

with open("letter.txt") as f:
    text = f.read()
#切分文本
    text_splitter = CharacterTextSplitter(
        chunk_size = 20,
        chunk_overlap = 5
    )
    texts = text_splitter.split_text(text)

#使用openai的embedding
    embddings = OpenAIEmbeddings()
    #使用chroma向量存储
    docssearch = Chroma.from_texts(
        texts,
        embddings,
    )
    query = "公司有什么新策略?"
    docs = docssearch.similarity_search(query=query)

In [ ]:
#构建问答对话链
from langchain.chains.question_answering import  load_qa_chain
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

llm = OpenAI(
    temperature=0,
)

template = """下面是一段AI与人类的对话，AI会针对人类问题，提供尽可能详细的回答，如果AI不知道答案，会直接回复'人类老爷，我真的不知道'，参考一下相关文档以及历史对话信息，AI会据此组织最终回答内容.
{context}
{chat_history}
Human:{human_input}
AI:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "chat_history", "human_input"],
)

#使用ConversationBufferMemory对对话进行缓存 
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="human_input",
    return_messages=True,
)

#加载对话链
chain = load_qa_chain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True,
    chain_type="stuff"
)

# 传入多个参数
chain({"input_documents":docs,"human_input":"公司的营销策略是什么？"})
